In [1]:
import TAB_generator
from TAB_generator import *
import os
path = os.getcwd()

In [2]:
p, f, t = readTAB(path + '\\test\\', show_tuning=True)

#####0. a[Air]鸟之诗1^9I.xml
special tuning:[0, 0, 0, 0, 0, 0], octave-change:0
#####1. a[圣斗士]原谅我 1^14A.xml
special tuning:[0, 0, 0, 0, 0, 0], octave-change:0
#####2. A[日在校园]悲しみの向こうへ(向着悲伤的彼方)18^26B.xml
special tuning:[-12, -12, -12, -12, -12, -12], octave-change:-1
#####3. a[植物大战僵尸]Grasswalk1^8A.xml
special tuning:[-14, -12, -12, -12, -12, -12], octave-change:-1
#####4. a[植物大战僵尸]Graze the Roof11^18A.xml
special tuning:[-14, -12, -12, -12, -12, -12], octave-change:-1
#####5. a[植物大战僵尸]植物保龄球 1^13A.xml
special tuning:[0, 0, 0, 0, 0, 0], octave-change:0
#####6. a[猎人]魂魄之挽歌C调 10^17A.xml
special tuning:[0, 0, 0, 0, 0, 0], octave-change:0
    ! In measure 0, sum of timesig should be 3.0, but6.0
    ! In measure 1, sum of timesig should be 3.0, but6.0
    ! In measure 2, sum of timesig should be 3.0, but6.0
    ! In measure 3, sum of timesig should be 3.0, but6.0
    ! In measure 4, sum of timesig should be 3.0, but6.0
    ! In measure 5, sum of timesig should be 3.0, but6.0
    ! In measure 6, sum

In [3]:
print(p[0][1])
print(f[0][1])

['4B 4E 4C', '3G', '4G', '4A', '5C']
['(1,7) (2,5) (3,5)', '(4,5)', '(2,8)', '(1,5)', '(1,8)']


In [4]:
all_chord = []
for songnum, song in enumerate(p):
    for measurenum, measure in enumerate(song):
        for pitchnum, pitchcu in enumerate(measure):
            if len(pitchcu.split()) >= 3 and int(f[songnum][measurenum][pitchnum].split()[0][1])>1:
                all_chord.append((pitchcu, f[songnum][measurenum][pitchnum]))
print(len(all_chord))               

228


In [5]:
import random
all_chord = random.sample(all_chord, 128)

In [6]:
all_chord

[('5A 5E 4F# 4C#', '(2,10) (3,9) (5,9) (6,9)'),
 ('4C 3G 3E', '(2,1) (3,0) (4,2)'),
 ('4F 4D 3A#', '(2,6) (3,7) (4,8)'),
 ('3A 3F# 2F#', '(3,2) (4,4) (6,2)'),
 ('3B 3G 3D', '(2,0) (3,0) (4,0)'),
 ('4C 3A 3F', '(2,1) (3,2) (4,3)'),
 ('3B 3G 3D 2G', '(2,0) (3,0) (4,0) (6,3)'),
 ('4C 3G 2G', '(2,1) (3,0) (6,3)'),
 ('4C 3A 3F', '(2,1) (3,2) (4,3)'),
 ('5A 5E 5C# 4F#', '(2,10) (3,9) (4,11) (5,9)'),
 ('3B 3G 2E', '(2,0) (3,0) (6,0)'),
 ('5D 4D 3G', '(2,3) (4,0) (6,3)'),
 ('3B 3F# 2B', '(3,4) (4,4) (5,2)'),
 ('4C 3G 2G', '(2,1) (3,0) (6,3)'),
 ('4D 3A 2F', '(2,3) (3,2) (6,1)'),
 ('4C 3G 3E 3C', '(2,1) (3,0) (4,2) (5,3)'),
 ('5B 5F# 5D 4G', '(2,12) (3,11) (4,12) (5,10)'),
 ('4C 3G# 3D# 3C', '(2,1) (3,1) (4,1) (5,3)'),
 ('3B 3G 3D 2G', '(2,0) (3,0) (4,0) (6,3)'),
 ('4C 3G# 3D# 3C', '(2,1) (3,1) (4,1) (5,3)'),
 ('4G# 4D 3E', '(2,9) (3,7) (5,7)'),
 ('3B 3G 2E', '(2,2) (3,0) (6,2)'),
 ('4D 3G 3E', '(2,3) (3,0) (4,2)'),
 ('3B 3G 3D', '(2,0) (3,0) (4,0)'),
 ('3A 3E 2B', '(3,2) (4,2) (5,2)'),
 ('4C 3

In [7]:
pitchset, fingerset, timesigset = [], [], []

count = 0
pitchmeasure = []
fingermeasure = []
for chord in all_chord:
    count += 1
    pitchmeasure.append(chord[0])
    fingermeasure.append(chord[1])
    
    if count == 8:
        count=0
        pitchset.append(pitchmeasure)
        fingerset.append(fingermeasure)
        pitchmeasure = []
        fingermeasure = []
        
for i in range(16):
    timesigset.append([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
generateTAB(pitchset,fingerset,  timesigset, filepath=path + '\\head.xml')

文件名为：head.xml
文件已保存为C:\Users\28056\Desktop\研究生\AnimeTAB\AnimeTAB\new_head.xml


In [8]:
generateTAB(p[0], f[0], t[0], filepath=path + '\\head.xml')

文件名为：head.xml
文件已保存为C:\Users\28056\Desktop\研究生\AnimeTAB\AnimeTAB\new_head.xml


In [9]:
#chord_recognize accurate test
cp, cf, ct = readTAB(path + '\\test\\chord_recognize test\\')
recognize_result = []
_ = []
for chord in split(cp[0]):
    _.append(chord_recognize(chord))
    
    if len(_) == 8:
        recognize_result.append(_)
        _ = []
        

#####0. chord_recognize test.xml
    ! In measure 16, sum of timesig should be 4.0, but0


[[('C', 'maj'),
  ('C', 'maj'),
  ('F', 'maj'),
  ('E', 'm'),
  ('F#', 'maj'),
  ('C', 'maj'),
  ('F#', 'maj'),
  ('A', 'm7')],
 [('A#', 'maj'),
  ('F', 'maj'),
  ('D', 'maj'),
  ('C', 'maj'),
  ('D', 'maj'),
  ('C', 'maj'),
  ('G', 'maj'),
  ('C', 'maj')],
 [('C', 'maj'),
  ('G', 'maj'),
  ('C', 'maj'),
  ('G', 'maj'),
  ('C', 'sus2'),
  ('G', 'maj'),
  ('E', 'm7'),
  ('F#', 'm7')],
 [('C', 'maj'),
  ('E', 'm'),
  ('E', 'm'),
  ('E', '7'),
  ('D', 'maj'),
  ('A', 'm'),
  ('C', 'maj'),
  ('G', 'maj')],
 [('G', 'maj'),
  ('G', 'maj'),
  ('C', 'maj'),
  ('A', 'maj'),
  ('B', 'maj'),
  ('B', 'm7'),
  ('F', 'maj'),
  ('F#', 'm7')],
 [('E', 'm'),
  ('F#', 'm7'),
  ('F#', 'm'),
  ('F', 'maj'),
  ('E', 'm'),
  ('C', 'maj'),
  ('D', 'm'),
  ('NotChord', 'NotChord')],
 [('A#', 'maj'),
  ('E', 'maj'),
  ('C', 'maj'),
  ('G', 'maj'),
  ('F', 'maj'),
  ('C', 'maj'),
  ('E', 'm'),
  ('A', 'maj')],
 [('C', 'maj7'),
  ('B', 'maj'),
  ('NotChord', 'NotChord'),
  ('E', 'm7'),
  ('C', 'maj'),
  ('B', 'm

In [25]:
import pandas as pd
yes, no = 0, 0 
truth = pd.read_csv('chord_recognize test.csv')
for num1, measure in enumerate(recognize_result):
    for num2, chord in enumerate(measure):
        real = truth.iloc[num1][num2]
        root, _type = 0, 0
        if '#' in real:
            root = real[:2]
            _type = real[2:]
        else:
            root = real[:1]
            _type = real[1:]
        if _type == '':
            _type = 'maj'
        #print("/{}/, {}".format(root, _type))
        if real != 'N' and (chord[0] + chord[1] != root+_type):
            print('measure{}, note{} wrong: should `` {} ``, but`` {} ``'.format(num1, num2, root+_type, chord[0] + chord[1]))
            no += 1
        elif chord[0] == root and chord[1] == _type and real != 'N':
            yes += 1
    print('=================================')
            
        

measure2, note4 wrong: should `` Gsus ``, but`` Csus2 ``
measure5, note7 wrong: should `` Gsus ``, but`` NotChordNotChord ``
measure7, note2 wrong: should `` Em7 ``, but`` NotChordNotChord ``
measure8, note0 wrong: should `` Asus ``, but`` NotChordNotChord ``
measure8, note3 wrong: should `` Dsus ``, but`` Gsus2 ``
measure10, note3 wrong: should `` Dsus ``, but`` Gsus2 ``
measure12, note6 wrong: should `` Bsus4 ``, but`` NotChordNotChord ``
measure14, note4 wrong: should `` Asus ``, but`` NotChordNotChord ``
measure15, note5 wrong: should `` Asus ``, but`` NotChordNotChord ``
measure15, note7 wrong: should `` Csus ``, but`` Csus2 ``


In [11]:
root

'C'

In [12]:
truth

,0,1,2,3,4,5,6,7
0,C,C,F,Em,F#,C,F#,Am7
1,A#,F,D,C,D,C,G,C
2,C,G,C,G,Gsus,G,Em7,F#m7
3,C,Em,Em,E7,D,Am,C,G
4,G,G,C,A,B,Bm7,F,F#m7
5,Em,F#m7,F#m,F,Em,C,Dm,Gsus
6,A#,E,C,G,F,C,Em,A
7,Cmaj7,B,Em7,Em7,C,Bm7,C,G
8,Asus,G,D#m7,Dsus,Em7,B,C,F#
9,N,A,D,N,F,G,F#m7,G#


In [13]:
def chord_recognize(pitch_cu, return_overtone=False):
    '''
    大三和弦：有且仅有大三度和五度， 435 435 435 或 75 或 48 39
    小三和弦：有且仅有小三度和五度， 345 345 345
    大七和弦：存在大七度和大三度， 4341 4341 4341或471
    小七和弦：存在小七度和小三度， 3432 3432 3432或732
    属七和弦：存在小七度和大三度， 4332 4332 4332或732
    增三和弦：存在增五度（小六度）， 444 444 444
    减三和弦：存在增四度， 326 326 326或
    挂留2和弦：存在二度但不存在三度， 255 255 255
    挂留4和弦：存在四度但不存在三度， 525 525 525
    '''
    in_chord_pitchs = [i[1:] for i in pitch_cu.split()][-4:]
    pitchs = list(set(in_chord_pitchs))
    singles = list(np.sort([str2midi(i) for i in pitchs]))
    print(singles)
    _ = singles
    singles.extend([i + 12 for i in _])
    singles.extend([i + 24 for i in _])
    
    delta_singles = []
    for i in range(len(singles)):
        if i > 0:
            delta_singles.append(singles[i] - singles[i-1])
    inverts = delta_singles
    feasure = ''
    for num in delta_singles:
        feasure += str(num)
    def ifin(str1):
        strlen = len(str1)
        newstrs = []
        for i in range(len(str1)):
            str1 = str1[1:] + str1[0]
            newstrs.append(str1)
        for newstr in newstrs:
            if newstr in feasure:
                return True
        return False
    print(feasure)
    chord_type = 'NotChord'
    chordchar = 'NotChord'
    if ifin('255'): #当一阶差分只有5半音和2半音时 说明是挂2
        chord_type = 'sus2'
        
        chordchar = midi2str(singles[feasure.index('2')])[1:]#2 前面的音符是挂2的根音

    elif ifin('336'):
        chord_type =  'dim'
        chordchar = midi2str(singles[feasure.index('6')+1])[1:]#6后面的是根音
    elif ifin('444'): #aug时返回最低的音
        chord_type = 'aug'
        chordchar = in_chord_pitchs[0]
    #七和弦组：
    
    if ifin('2') or ifin('1'): 
        #属七：
        if ifin('4332') or ifin('246'):
            chord_type = '7'
            chordchar = midi2str(singles[feasure.index('4')])[1:] #4前面
        #小七：              
        elif ifin('3432') or ifin('723') or ifin('732'):
            chord_type = 'm7'
            chordchar = midi2str(singles[feasure.index('2')+1])[1:] #2后面
        elif ifin('4341') or ifin('471') or ifin('741'):
            chord_type = 'maj7'
            chordchar = midi2str(singles[feasure.index('1')+1])[1:] #1后面
    #三和弦组：
    else:
        if ifin('345') or ifin('39'):
            chord_type = 'm'
            chordchar = midi2str(singles[feasure.index('3')])[1:] #3前面
        elif ifin('435') or ifin('75'):
            chord_type = 'maj'
            chordchar = midi2str(singles[feasure.index('5')+1])[1:] #5后面
        elif ifin('48') :
            chord_type = 'maj'
            chordchar = midi2str(singles[feasure.index('4')])[1:] #4前面
    if return_overtone == True:
        return [chord_type, singles]
    else:
        return chordchar, chord_type
    
    

In [26]:
print(yes, no)

110 10


In [15]:
print(cp[0][3][5])

5C 4A 3A


In [16]:
chord_recognize(cp[0][7][2])

[74, 76, 79, 81]
232523252325232


('NotChord', 'NotChord')

In [17]:
char, _type = chord_recognize('4D 4F 4G#')
print(char, _type)

[74, 77, 80]
33633633633
D dim


In [18]:
a = '255'
a.index('5')

1